In [1]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI()

In [657]:
def  geography_question(question, answers):
    print("\nChoose the correct answer!\n")
    print(question)
    print('\n')
    for num, ans in enumerate(answers):
        print(f"Option {num+1}.): {ans}")

    print("\n")
    choice = input("Your choice: ")
    print(answers[int(choice) - 1])
          
    return answers[int(choice) - 1]

In [747]:
geography_question("What is the capital of Hungary?",
    ["Vilnius", "Bishkek", "Budapest", "Taskent"]
)


Choose the correct answer!

What is the capital of Hungary?


Option 1.): Vilnius
Option 2.): Bishkek
Option 3.): Budapest
Option 4.): Taskent




Your choice:  3


Budapest


'Budapest'

In [679]:
function_json = {'type': 'function',
                 'function':{
                     'name':'geography_question',
                     'description': 'Prints out a Geography question and the possible answers to choose from. Asks the user to input an answer and returns the given answer.
                     'parameters':{
                         'type':'object',
                         'properties':{
                             'question':{'type':'string',
                                         'description': 'A Geography-related test question'
                                        },
                            'answers':{'type': 'array',
                                       'items': {'type': 'string'},
                                       'description': '4 answers to choose from, with only one correct. Include purely the answer strings and dont include any list markings.'
                                        }
                         },
                         'required': ['question','answers']
                     }
                 }
                }


In [680]:
instructions = """You help create a Geography quiz, providing a question and 4 possible answers for the user to 
                choose from, with only one of them correct. The user responds with an answer, you evaluate if 
                the user's response is correct."""

In [681]:
# Create the assistant
assistant = client.beta.assistants.create(
    name="Geography Quiz",
    instructions=instructions,
    tools = [function_json],
    model = 'gpt-3.5-turbo'
)

In [714]:
client.beta.threads.delete(thread.id)

ThreadDeleted(id='thread_FKscHo2C4s1J27T3ejdeUEMe', deleted=True, object='thread.deleted')

In [715]:
# Create a thread
thread = client.beta.threads.create()

In [716]:
thread.id

'thread_93cCjExfuosnTvYGpMBiBpPO'

In [717]:
# Create our first message.
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content = "Create a new Geography quiz question. Tell the user if their answer was correct or incorrect, and tell the correct answer if the given answer was incorrect."
)

In [718]:
run = client.beta.threads.runs.create(
        assistant_id = assistant.id,
        thread_id = thread.id)

In [719]:
# Create a run for the thread
import time

def run_and_wait(run, thread):
    while run.status == 'queued' or run.status == 'in_progress':
        run = client.beta.threads.runs.retrieve(
            thread_id = thread.id,
            run_id = run.id)
        print(run.status)
        time.sleep(2)

    # requires_action
    return run

In [720]:
run = run_and_wait(run, thread)

requires_action


In [721]:
run

Run(id='run_NU5VZinJg8HSeNTAmvAVmv5B', assistant_id='asst_QKNrc7ZKPZakQP6EV16kSXyA', cancelled_at=None, completed_at=None, created_at=1718527957, expires_at=1718528557, failed_at=None, incomplete_details=None, instructions="You help create a Geography quiz, providing a question and 4 possible answers for the user to \n                choose from, with only one of them correct. The user responds with an answer, you evaluate if \n                the user's response is correct.", last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', parallel_tool_calls=True, required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_sFVtJ23FGaECcUlX1oDfg1eJ', function=Function(arguments='{"question":"Which is the largest desert in the world by area?","answers":["Sahara Desert","Arabian Desert","Gobi Desert","Kalahari Desert"]}', name='geography_question'), typ

In [722]:
run.required_action

RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_sFVtJ23FGaECcUlX1oDfg1eJ', function=Function(arguments='{"question":"Which is the largest desert in the world by area?","answers":["Sahara Desert","Arabian Desert","Gobi Desert","Kalahari Desert"]}', name='geography_question'), type='function')]), type='submit_tool_outputs')

In [723]:
run.required_action.submit_tool_outputs

RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_sFVtJ23FGaECcUlX1oDfg1eJ', function=Function(arguments='{"question":"Which is the largest desert in the world by area?","answers":["Sahara Desert","Arabian Desert","Gobi Desert","Kalahari Desert"]}', name='geography_question'), type='function')])

In [724]:
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]

In [725]:
import json
arguments = json.loads(tool_call.function.arguments)
function_name = tool_call.function.name

In [726]:
arguments

{'question': 'Which is the largest desert in the world by area?',
 'answers': ['Sahara Desert',
  'Arabian Desert',
  'Gobi Desert',
  'Kalahari Desert']}

In [727]:
function_name

'geography_question'

In [728]:
# We run the required action
response = python_question(arguments['question'], arguments['answers'])


Choose the correct answer!

Which is the largest desert in the world by area?


Option 1: Sahara Desert
Option 2: Arabian Desert
Option 3: Gobi Desert
Option 4: Kalahari Desert




Your choice:  1


Sahara Desert


In [729]:
response

'Sahara Desert'

In [730]:
# We submit the output via submit_tool_output to be used by the assistant, who still has to decide if the given
# answer was correct.
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id = thread.id,
    run_id = run.id,
    tool_outputs = [{'tool_call_id': tool_call.id,
                     'output': json.dumps(response)
                    }]
)

In [731]:
run = run_and_wait(run, thread)

completed


In [732]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
    order = 'asc'
)

In [733]:
for message in messages:
    print(message.content[0].text.value)
    print('\n')

Create a new Geography quiz question. Tell the user if their answer was correct or incorrect, and tell the correct answer if the given answer was incorrect.


The user's answer "Sahara Desert" is correct! The Sahara Desert is indeed the largest desert in the world by area. Great job!




In [734]:
# To continue with the same thread, we create a new message and repeat all the steps from that point.
# I didn't put this functionality into a loop here to better understand how the function calling works.

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content = "Create a new Geography quiz question. Tell the user if their answer was correct or incorrect, and tell the correct answer if the given answer was incorrect."
)

In [735]:
run = client.beta.threads.runs.create(
        assistant_id = assistant.id,
        thread_id = thread.id)

In [736]:
run = run_and_wait(run, thread)

requires_action


In [737]:
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]

In [738]:
arguments = json.loads(tool_call.function.arguments)

In [739]:
response = python_question(arguments['question'], arguments['answers'])


Choose the correct answer!

Which country is known as the Land of the Rising Sun?


Option 1: Japan
Option 2: China
Option 3: South Korea
Option 4: Thailand




Your choice:  1


Japan


In [740]:
response

'Japan'

In [741]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id = thread.id,
    run_id = run.id,
    tool_outputs = [{'tool_call_id': tool_call.id,
                     'output': json.dumps(response)
                    }]
)

In [742]:
run = run_and_wait(run, thread)

completed


In [743]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
    order = 'asc'
)

In [744]:
for message in messages:
    print(message.content[0].text.value)
    print('\n')

Create a new Geography quiz question. Tell the user if their answer was correct or incorrect, and tell the correct answer if the given answer was incorrect.


The user's answer "Sahara Desert" is correct! The Sahara Desert is indeed the largest desert in the world by area. Great job!


Create a new Geography quiz question. Tell the user if their answer was correct or incorrect, and tell the correct answer if the given answer was incorrect.


The user's answer "Japan" is correct! Japan is indeed known as the Land of the Rising Sun. Well done!




In [745]:
# For debugging:
run_steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id
)

In [746]:
for step in run_steps:
    print(step.step_details)
    print('\n')

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_jPvuZBEqDPMGDXL2sFV7z8BP'), type='message_creation')


ToolCallsStepDetails(tool_calls=[FunctionToolCall(id='call_6g9YpWRucBoHZiWkHRb1GmjC', function=Function(arguments='{"question":"Which country is known as the Land of the Rising Sun?","answers":["Japan","China","South Korea","Thailand"]}', name='geography_question', output='"Japan"'), type='function')], type='tool_calls')




In [678]:
# Delete the assistant:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_tD95RPQx1hNl1pChSjAl8Z19', deleted=True, object='assistant.deleted')